<a href="https://colab.research.google.com/github/zenon10/ria_codes_vf/blob/main/std_non_std_clause_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [65]:
import pandas as pd
import json
import os

In [23]:
# df = pd.read_csv('/content/drive/MyDrive/Zenon_POC_OCR/Clause Management_Deep Dive France_Mapping Document_key words.csv')

In [26]:
# clause = """Dans le délai de paiement sont incluses les phases de réception de la commande et de certification du 
#                     service fait par le service gestionnaire, de visa de la dépense (appelé aussi le mandatement) et de 
#                     règlement par le comptable public. En cas de dépassement du délai de paiement, des intérêts moratoires 
#                     sont versés au titulaire, calculés par application de la formule suivante : IM = M x J/365 x Taux IM + F ; 
#                     Dans laquelle : IM : montant des intérêts moratoires - M : montant TTC de la demande de paiement - Taux IM :
#                     taux de la Banque Centrale Européenne en vigueur majoré de 8 points - J : nombre de jours calendaires entre 
#                     la date limite et la date réelle de paiement - F : forfait de 40 € de frais de recouvrement """


In [75]:
def std_clause(df, clause):

    std = []

    for indx, stdd in enumerate(df['std/non-std'], 0):
        if stdd == 'std':
            keywords = df['list of keywords for std/non-std'][indx].split(',')

            if len(keywords) > 1:
                count = 0
                for k in keywords[:-1]:
                    if clause.__contains__(k):
                        # print(k)
                        count = count + 1
                        std.append(stdd) if count == len(keywords[:-1]) else ''
            else:
                for k in keywords:
                    if clause.__contains__(k):
                        std.append(stdd)
            
    return std

In [76]:
def non_std_clause(df, clause):
    
    non_std = []
    for indx, nonstd in enumerate(df['std/non-std'], 0):
        if nonstd == 'non-std':
            keywords = df['list of keywords for std/non-std'][indx].split(',')

            if len(keywords) > 1:
                count = 0
                for k in keywords[:-1]:
                    if clause.__contains__(k):
                        # print(k)
                        count = count + 1
                        non_std.append(nonstd) if count != 0 else ''
                    else:
                        for k in keywords:
                            if clause.__contains__(k):
                                non_std.append(nonstd)

    return non_std




In [81]:
def classify_std_nonstd_wrapper(input_file_path, input_json_path, output_folder_path): ## put metadata input file path and call json path from there

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path )

    df = pd.read_csv(input_file_path)
    df=df.fillna('')

    # import json
    # Opening JSON file
    with open(input_json_path) as json_file:
        data = json.load(json_file)

        for i in data['clauses']:
            if i['clause'] in ['samples_tc','shipment_and_delivery_requirements_tc','sustainability_tc','demo_session_tc', 'list_of_documents_required_tc', 'delivery_time_tc','delivery_specifications_tc', 'capital_equipment_loan_contract_tc','maintenance_contract_tc', 'delay_penalties_(supply)_tc', 'delay_penalties_for_claims_management_tc','signing_required_tc', 'renewal_clause_tc', 'tender_description_tc']:
                i['classify_std/non-std'] = 'non-std'
                print(i['clause'])
            else:
                # Calling std function
                std = std_clause(df, i['value'])
                i['classify_std/non-std'] = std 
                
                if not std:
                    print('Empty list')
                    non_std = non_std_clause(df, i['value'])
                    print(non_std)
                    i['classify_std/non-std'] = non_std
                else:
                    print('List is not Empty \n',std)
            
    with open(output_folder_path + '/nameofdoc.json', 'w', encoding = 'utf-8') as f: # check once for utf
        json.dump(data, f, indent=2)
## make column B anfd f combination



In [82]:
input_file_path = '/content/drive/MyDrive/Zenon_POC_OCR/Clause Management_Deep Dive France_Mapping Document_key words.csv'
input_json_path = '/content/drive/MyDrive/Zenon_POC_OCR/clauses_response_flat_format_new.json'
output_folder_path = '/content/drive/MyDrive/Zenon_POC_OCR/output_std_nonstd'

classify_std_nonstd_wrapper(input_file_path, input_json_path, output_folder_path)


samples_tc
Empty list
[]
Empty list
[]
Empty list
[]
Empty list
[]
Empty list
[]
Empty list
[]
List is not Empty 
 ['std']
List is not Empty 
 ['std']
sustainability_tc


In [52]:
# clause_type = []
# for i in ['Samples','Shipment And Delivery Requirements','Sustainability','Demo session','List of documents required','Delivery time','Delivery specifications',
#              'Capital equipment loan contract','Maintenance contract','Delay penalties (supply)','Delay penalties for claims management','Signing required','Renewal clause',
#              'Tender Description']:
#              i = i.lower()
#              i = i.split()
#              i = "_".join(i)
#              i = i + '_tc'
#              clause_type.append(i)